**Programming with Applied AI**

Start by adding the easyocr services to our virtual environment

In [ ]:
# easyocr is a low dependency, highly efficient computer vision aligned module
# able to handle a wide variety of character types, fonts etc
!pip install easyocr # 'preferred installer program', add easyocr module to our environment

<br>
Add the required libraries to our OCR program

In [ ]:
import easyocr # add the ocr module to the project

import matplotlib.pyplot as plt # visualisation library
import matplotlib.patches as patches # extends library to include image overlays

from PIL import Image # add 'python imaging lib' to our project
import numpy as np # add support for multi-dimension arrays/matrices

import pprint # “pretty-print” Python data structures
pp = pprint.PrettyPrinter(indent=4) # set a default

In [ ]:
# create a simple function to calculate coordinated
def extract_boundary(textbox):
  TOPLEFT = 0 # set constants for each textbox boundary
  TOPRIGHT = 1
  BOTTOMRIGHT = 2
  BOTTOMLEFT = 3

  x, y = textbox[BOTTOMLEFT] # calculate coords
  x1, y1 = textbox[TOPRIGHT]

  w = x1 - x # calculate width & height
  h = y1 - y

  return x, y, w, h


<br>To make things a ltlle easier, lets initialise a manual file upload

In [ ]:
from google.colab import files # add support to upload files
uploaded = files.upload() # initialise a manual file upload

<br>Iterate the through the file (or file in our case); the benefit of this approach is you don't need to remember the name (or location) where the file was uploaded

In [ ]:
for file in uploaded.keys():

    image = Image.open(file) # use the PIL module to load the specified file
    fig, ax = plt.subplots() # create a figure and axis; essentially controls plots
    #ax.imshow(image) # render image

    reader = easyocr.Reader(['en'], gpu = True) # initialise model for english (en) and gpu's
    result = reader.readtext(file) # process the (uploaded) file using the AI reader and store the results

    # repeat for each text string found
    for text in result:
      x, y, w, h = extract_boundary(text[0]) # use the text coordinates to calculate the boundary box

      # update the image to demonstrate the found text strings
      colour = (np.random.random(), np.random.random(), np.random.random())
      boundary = patches.Rectangle( (x, y), w, h, linewidth = 1, edgecolor = colour, facecolor = 'none')
      ax.add_patch(boundary)

    # show a preview of the image
    ax.imshow(image) # render image
    plt.axis("off")
    plt.show()

In [ ]:
# outout the found data
print("\n\nRaw Results")
pp.pprint(result)


<br>Lets review what was recognised


In [ ]:
for text in result:
  print("\nCoordinates: ", text[0])
  print("Recognised Characters: ", text[1])
  print("Confidence %: ", text[2])

print()

<br>Lets filter the strings based on the percentage confidence

In [ ]:
for text in result: # as before, iterate through the results
  if text[2] > 0.9: # only 'print' strings with at least a 90% confidence
    print("Recognised Characters: ", text[1], end='')
    print(", ", f"{text[2]:.2%}")
